In [1]:
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langchain_community.document_loaders import TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.llms import Ollama
from langchain.prompts import ChatPromptTemplate
from langchain.chains import RetrievalQA
from langchain.schema import StrOutputParser
from langchain_core.tracers import LangChainTracer
import os

In [3]:
os.environ["LANGCHAIN_API_KEY"]  
os.environ["LANGCHAIN_PROJECT"]      

'GenAI'

In [4]:
# Tracker
tracer = LangChainTracer()

In [5]:
# Loading data
loader = TextLoader("data.txt") 
docs = loader.load()

In [6]:
# Splitting into smaller chunks
splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
splits = splitter.split_documents(docs)

In [7]:
# Creating embeddings for chunks
embedding = OllamaEmbeddings(model="nomic-embed-text") 
vectordb = Chroma.from_documents(documents=splits, embedding=embedding, persist_directory="./chroma_db")
vectordb.persist()

C:\Users\BW\AppData\Local\Temp\ipykernel_21912\384518551.py:2: LangChainDeprecationWarning: The class `OllamaEmbeddings` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaEmbeddings``.
  embedding = OllamaEmbeddings(model="nomic-embed-text")
C:\Users\BW\AppData\Local\Temp\ipykernel_21912\384518551.py:4: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [8]:
# Creating retriever
retriever = vectordb.as_retriever()

In [9]:
# Setting up the propmt
prompt = ChatPromptTemplate.from_template(
    """You are a Philosopher AI assistant. Use the following context to answer the user's question.
    If you don't know the answer, say you don't know.

    Context: {context}

    Question: {question}
    """
)

In [10]:
# Setting up the LLM
llm = Ollama(model="gemma3:1b")

C:\Users\BW\AppData\Local\Temp\ipykernel_21912\1674934915.py:2: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="gemma3:1b")


In [11]:
# Creating the chain with tracing
qa_chain = RetrievalQA.from_chain_type(llm=llm, retriever=retriever, chain_type_kwargs={"prompt": prompt}, callbacks=[tracer])

In [12]:
print("\nPhilosopher AI is ready. Ask your questions!\n")
while True:
    try:
        query = input("Ask (or 'exit' to quit): ").strip()
        if query.lower() == "exit":
            print("\nExiting... Goodbye!")
            break

        if not query:
            print("Please enter a valid question.\n")
            continue

        result = qa_chain.invoke({"query": query})
        print("\nAnswer:", result["result"], "\n")

    except Exception as e:
        print(f"\nAn error occurred: {e}\n")


Philosopher AI is ready. Ask your questions!


Answer: Philosophy is the art of questioning and exploring fundamental truths about existence, reason, and our understanding of the world. It seeks wisdom and helps us navigate life’s complexities. 


Exiting... Goodbye!
